In [ ]:
print("Connecting to Weaviate...")
client = weaviate.connect_to_local(
    host="localhost",
    port=8080,
    grpc_port=50051,
    headers={"X-Cohere-Api-Key": COHERE_API_KEY},
    skip_init_checks=True  
)


Connecting to Weaviate...


In [ ]:
import json
import base64
import weaviate
import weaviate.classes as wvc
import requests
from PIL import Image
from io import BytesIO

COHERE_API_KEY = <COHERE_API_KEY> ##########ADD COHERE_API_KEY HERE 

foods_data_extended = [
    {"name": "Margherita Pizza", "description": "Classic Italian pizza with tomato sauce, mozzarella, and fresh basil", "price": 12.99, "calories": 800, "image_url": "https://images.unsplash.com/photo-1574071318508-1cdbab80d002?w=400"},
    {"name": "Pepperoni Pizza", "description": "Traditional pizza topped with spicy pepperoni slices and melted cheese", "price": 14.99, "calories": 950, "image_url": "https://images.unsplash.com/photo-1628840042765-356cda07504e?w=400"},
    {"name": "Vegetarian Pizza", "description": "Healthy pizza loaded with bell peppers, mushrooms, olives, and onions", "price": 13.99, "calories": 720, "image_url": "https://images.unsplash.com/photo-1571997478779-2adcbbe9ab2f?w=400"},
    {"name": "Spaghetti Carbonara", "description": "Creamy Italian pasta with eggs, pancetta, parmesan cheese", "price": 15.99, "calories": 850, "image_url": "https://images.unsplash.com/photo-1612874742237-6526221588e3?w=400"},
    {"name": "Spaghetti Bolognese", "description": "Italian pasta with rich meat sauce and parmesan", "price": 14.99, "calories": 780, "image_url": "https://images.unsplash.com/photo-1621996346565-e3dbc646d9a9?w=400"},
    {"name": "Fettuccine Alfredo", "description": "Creamy pasta with butter, cream, and parmesan cheese", "price": 13.99, "calories": 920, "image_url": "https://images.unsplash.com/photo-1645112411341-6c4fd023714a?w=400"},
    {"name": "Caesar Salad", "description": "Fresh romaine lettuce with caesar dressing, croutons, and parmesan", "price": 9.99, "calories": 350, "image_url": "https://images.unsplash.com/photo-1546793665-c74683f339c1?w=400"},
    {"name": "Greek Salad", "description": "Mediterranean salad with feta cheese, olives, tomatoes, and cucumber", "price": 10.99, "calories": 280, "image_url": "https://images.unsplash.com/photo-1540189549336-e6e99c3679fe?w=400"},
    {"name": "Cheeseburger", "description": "Juicy beef patty with melted cheese, lettuce, tomato, and pickles", "price": 11.99, "calories": 650, "image_url": "https://images.unsplash.com/photo-1568901346375-23c9450c58cd?w=400"},
    {"name": "Bacon Burger", "description": "Classic burger with crispy bacon, cheese, and special sauce", "price": 13.99, "calories": 780, "image_url": "https://images.unsplash.com/photo-1553979459-d2229ba7433b?w=400"},
    {"name": "Sushi Platter", "description": "Assorted fresh sushi rolls with salmon, tuna, and avocado", "price": 18.99, "calories": 450, "image_url": "https://images.unsplash.com/photo-1579584425555-c3ce17fd4351?w=400"},
    {"name": "Salmon Sashimi", "description": "Fresh sliced salmon served with soy sauce and wasabi", "price": 16.99, "calories": 320, "image_url": "https://images.unsplash.com/photo-1617196034796-73dfa7b1fd56?w=400"},
    {"name": "Chocolate Cake", "description": "Rich and moist chocolate layer cake with chocolate frosting", "price": 6.99, "calories": 420, "image_url": "https://images.unsplash.com/photo-1578985545062-69928b1d9587?w=400"},
    {"name": "Cheesecake", "description": "Creamy New York style cheesecake with graham cracker crust", "price": 7.99, "calories": 380, "image_url": "https://images.unsplash.com/photo-1533134242443-d4fd215305ad?w=400"},
    {"name": "Tiramisu", "description": "Classic Italian dessert with coffee-soaked ladyfingers and mascarpone", "price": 8.99, "calories": 450, "image_url": "https://images.unsplash.com/photo-1571877227200-a0d98ea607e9?w=400"},
    {"name": "Pad Thai", "description": "Thai stir-fried rice noodles with shrimp, peanuts, and lime", "price": 13.99, "calories": 600, "image_url": "https://images.unsplash.com/photo-1559314809-0d155014e29e?w=400"},
    {"name": "Green Curry", "description": "Spicy Thai curry with coconut milk, vegetables, and chicken", "price": 14.99, "calories": 520, "image_url": "https://images.unsplash.com/photo-1455619452474-d2be8b1e70cd?w=400"},
    {"name": "Tacos", "description": "Three soft tortillas filled with seasoned beef, cheese, and salsa", "price": 10.99, "calories": 550, "image_url": "https://images.unsplash.com/photo-1565299585323-38d6b0865b47?w=400"},
    {"name": "Burrito Bowl", "description": "Mexican rice bowl with beans, meat, guacamole, and sour cream", "price": 12.99, "calories": 680, "image_url": "https://images.unsplash.com/photo-1626700051175-6818013e1d4f?w=400"},
    {"name": "French Fries", "description": "Crispy golden fries seasoned with salt", "price": 4.99, "calories": 365, "image_url": "https://images.unsplash.com/photo-1573080496219-bb080dd4f877?w=400"},
]

def download_image_to_base64(url):
    try:
        response = requests.get(url, timeout=10)
        img = Image.open(BytesIO(response.content))
        img = img.resize((400, 400))
        buffered = BytesIO()
        img.save(buffered, format="JPEG")
        return base64.b64encode(buffered.getvalue()).decode("utf-8")
    except Exception as e:
        print(f"Failed to download {url}: {e}")
        return None

print("Connecting to Weaviate...")
client = weaviate.connect_to_local(
    host="localhost",
    port=8080,
    headers={"X-Cohere-Api-Key": COHERE_API_KEY},
    skip_init_checks=True  
)

try:
    print("Is Weaviate ready?", client.is_ready())
    
    if client.collections.exists("FoodsMultiModal"):
        client.collections.delete("FoodsMultiModal")
        print("Old collection deleted")
    
    print("Creating collection...")
    client.collections.create(
        name="FoodsMultiModal",
        vectorizer_config=[
            wvc.config.Configure.NamedVectors.text2vec_cohere(
                name="text_vector",
                source_properties=["name", "description"]
            ),
            wvc.config.Configure.NamedVectors.multi2vec_clip(
                name="image_vector",
                image_fields=["image"]
            )
        ],
        properties=[
            wvc.config.Property(name="name", data_type=wvc.config.DataType.TEXT),
            wvc.config.Property(name="description", data_type=wvc.config.DataType.TEXT),
            wvc.config.Property(name="price", data_type=wvc.config.DataType.NUMBER),
            wvc.config.Property(name="calories", data_type=wvc.config.DataType.INT),
            wvc.config.Property(name="image", data_type=wvc.config.DataType.BLOB),
        ]
    )
    
    print("Collection created successfully!")
    print("\nUploading food data...")
    
    foods_collection = client.collections.get("FoodsMultiModal")
    
    for idx, food in enumerate(foods_data_extended, 1):
        print(f"[{idx}/{len(foods_data_extended)}] Processing {food['name']}...")
        
        image_base64 = download_image_to_base64(food['image_url'])
        
        if image_base64:
            obj = {
                "name": food["name"],
                "description": food["description"],
                "price": food["price"],
                "calories": food["calories"],
                "image": image_base64
            }
            
            try:
                uuid = foods_collection.data.insert(properties=obj)
                print(f"  SUCCESS: {uuid}")
            except Exception as e:
                print(f"  FAILED: {str(e)}")
        else:
            print(f"  SKIPPED: Image download failed")
    
    print("\nDatabase setup completed!")

except Exception as e:
    print(f"ERROR: {e}")

finally:
    client.close()
    print("Connection closed")


Connecting to Weaviate...


C:\Users\azza\AppData\Local\Temp\ipykernel_10736\3780155458.py:47: ResourceWarning: unclosed <socket.socket fd=1684, family=23, type=1, proto=0, laddr=('::1', 58260, 0, 0), raddr=('::1', 8080, 0, 0)>
  client = weaviate.connect_to_local(


Is Weaviate ready? True
Creating collection...
Collection created successfully!

Uploading food data...
[1/20] Processing Margherita Pizza...
  SUCCESS: 6c02d45c-4b7c-4f7c-87c1-f598aba0a1a7
[2/20] Processing Pepperoni Pizza...
  SUCCESS: 563ffa6b-1812-4ce8-8dc3-360a6d41fe82
[3/20] Processing Vegetarian Pizza...
  SUCCESS: 16c41f63-5d6c-4061-adf5-1df0cfe3a5b3
[4/20] Processing Spaghetti Carbonara...
  SUCCESS: 2d5d2308-fcf3-4b66-93dc-5c344b94d072
[5/20] Processing Spaghetti Bolognese...
  SUCCESS: 837e0bda-a7aa-4195-b94e-5696e182f54a
[6/20] Processing Fettuccine Alfredo...
  SUCCESS: 1b196c42-3256-4eb5-9ff7-eb0c309966ca
[7/20] Processing Caesar Salad...
  SUCCESS: 0bb726a4-651b-4a02-abce-b563c0e609cf
[8/20] Processing Greek Salad...
  SUCCESS: 9621f523-7464-4831-bab6-0f209fb46616
[9/20] Processing Cheeseburger...
  SUCCESS: 5ffd9589-6bc5-404f-a895-ca3106a064a2
[10/20] Processing Bacon Burger...
  SUCCESS: ed9c5910-dac1-42fb-b068-0098995f5d81
[11/20] Processing Sushi Platter...
  SUCCESS